# Project: digit recognizer
---

 Some say that the "Hello World!" of machine learning is building a classifier that recognizes 28x28 digits using the MNIST hand-written digit database. Sure, there are plenty of copy-and-paste tutorials out there, but here is my implementation where I have made an effort to write and understand every line within my implementation. 
 
This is a learning exercise for me, so there are plenty of notes inside to keep track of what I am doing and why. Enjoy!

##### Implementation details
- Implemented on Windows 10 (TF session setup may need to be tweaked for Linux run, see Python 3 'os' docs for details)
- Graphics card: GeForce GTX 1050 Ti

In [1]:
# Project setup
try:
    import os as os
    import sys as sys
    import keras as k
    from keras.backend import tensorflow_backend as tb
    from keras.layers import Dense, Dropout, Flatten, Conv2D, MaxPool2D
    from keras.models import Sequential
    from keras.utils.np_utils import to_categorical # for one-hot-encoding
    import matplotlib.pyplot as plt
    import numpy as np
    import pandas as pd
    import seaborn as sns
    from sklearn.naive_bayes import GaussianNB, MultinomialNB
    from sklearn.model_selection import train_test_split
    import tensorflow as tf
    print('Setup successful!')
except Exception as e:
    print("Setup failed:\n\t" + str(e))
    sys.exit(1)

Using TensorFlow backend.


Setup successful!


In [2]:
# TF session setup
random_seed = 2
np.random.seed(random_seed)
if tb._SESSION is None:
    if not os.environ.get('OMP_NUM_THREADS'):
#         print("os.environ.get('OMP_NUM_THREADS') not found.")
        config = tf.ConfigProto(allow_soft_placement=True)
    else:
#         print("TRUE OMP NUM THREADS, RESULT =", os.environ.get('OMP_NUM_THREADS'))
        num_thread = int(os.environ.get('OMP_NUM_THREADS'))
        config = tf.ConfigProto(intra_op_parallelism_threads=num_thread, allow_soft_placement=True)
    config.gpu_options.allow_growth=True
    _SESSION = tf.Session(config=config)
session = _SESSION

In [3]:
df = pd.read_csv("train.csv", dtype=np.float64)

In [4]:
labels = df.label.astype(np.int64).copy()
features = df.drop(labels=["label"], axis=1).copy() / 255.0
# print("Number of columns in 'features':", len(features.columns))

In [5]:
# Get a histogram of label distribution
# Uncomment this later
# sns.countplot(labels)

In [6]:
# features.values.reshape(-1, 28,28,1)

In [7]:
# one hot encode labels
# Reshape input pixel data from 1x784 to 28x28(x1)
features = features.values.reshape(-1,28,28,1)


# labels = labels.values.reshape(-1,28,28,1)
# Onehot encode labels
labels = to_categorical(labels, num_classes = 10)
# Flip it from a row vector to a column vector
labels = labels[:,0:1]
# sys.exit()

In [8]:
features_train, features_test, labels_train, labels_test = train_test_split(features, labels, test_size=0.1, random_state=random_seed)

In [17]:
# View a digit
# plt.imshow(features[0][:,:,0])

Keras models/layers details

- filters: dimensionality of output space
- kernel size: 

In [36]:
# Build the model
model = Sequential()

model.add(Conv2D(filters=28,kernel_size=(1,1),
                padding = 'Same', activation='relu',
                input_shape=(28,28,1)))
# model.add(Conv2D(filters=100,kernel_size=(5,5),
#                 padding = 'Same', activation='relu',
#                 input_shape=(28,28,1)))
model.add(Flatten())
model.add(Dense(10, activation='softmax'))

In [37]:
# Declare optimizer and compile
# lr=learning rate, 
# rho="Gradient M.A. decay factor",
# epsilon=fuzz factor,
# decay = delta lr per update,
optimizer = k.optimizers.RMSprop(lr=0.001, rho=0.9, epsilon=None, decay=0.0)
model.compile(optimizer=optimizer,
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])
# model.fit variables
epochs = 2
batch_size = 100

In [38]:
#     history = model.fit(features_train, labels_train, batch_size = batch_size, epochs = epochs, 
#              validation_data = (features_test, labels_test), verbose = 2)


model.fit(features_train, labels_train, epochs=epochs, batch_size=batch_size)
print()
loss_and_metrics = model.evaluate(features_test, labels_test) # WHAT IS THE DEFAULT BATCH SIZE?

#     print("\n", loss_and_metrics)
print("Accuracy: ",loss_and_metrics[1], " | Loss: ", loss_and_metrics[0], sep="")

Epoch 1/1
37800/37800 [==============================] - 7s 193us/step - loss: 0.0447 - acc: 0.9863

4200/4200 [==============================] - 1s 152us/step
Accuracy: 0.9871428571428571 | Loss: 0.03991178985876364
